# Introduction

You are tasked to address the task of POS tagging.

<center>
    <img src="images/pos_tagging.png" alt="POS tagging" />
</center>

# T1 - Corpus

Dataset: [Penn TreeBank corpus](https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip).

Note: **ignoring** the numeric value in the third column, use **only** the words/symbols and their POS label.

### Example

```Pierre	NNP	2
Vinken	NNP	8
,	,	2
61	CD	5
years	NNS	6
old	JJ	2
,	,	2
will	MD	0
join	VB	8
the	DT	11
board	NN	9
as	IN	9
a	DT	15
nonexecutive	JJ	15
director	NN	12
Nov.	NNP	9
29	CD	16
.	.	8
```

In [28]:
# file management
import os
import sys
import re
import shutil
import urllib
import tarfile
import zipfile
from pathlib import Path

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

### Corpus Download

In [4]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)

        
def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def extract_dataset(download_path: Path, extract_path: Path):
    print("Extracting dataset... (it may take a while...)")
    
    # Check if it's a zip file
    if download_path.suffix == '.zip':
        print("Extracting zipfile")
        with zipfile.ZipFile(download_path, 'r') as loaded_zip:
            loaded_zip.extractall(extract_path)
    
    # Check if it's a tar file
    elif download_path.suffix in ('.tar', '.gz', '.bz2', '.xz'):
        print("Extracting tarfile")
        with tarfile.open(download_path) as loaded_tar:
            loaded_tar.extractall(extract_path)
    
    else:
        print("Unsupported archive format. Please provide a zip or tar file.")
        return
    
    print("Extraction completed!")

In [5]:
ds = "dependency_treebank.zip"
dataset_url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dataset/dependency_treebank"

print(f"Current work directory: {Path.cwd()}")
folder_dataset = Path.cwd().joinpath("datasets")

if not folder_dataset.exists():
    folder_dataset.mkdir(parents=True)

dataset_tar_path = folder_dataset.joinpath(ds)
dataset_path = folder_dataset.joinpath(dataset_name)

if not dataset_tar_path.exists():
    download_dataset(dataset_tar_path, dataset_url)

if not dataset_path.exists():
    extract_dataset(dataset_tar_path, folder_dataset)

Current work directory: c:\Users\ardiz\OneDrive\Desktop\Projects\AIML-Research\ML Foundations\4 - NLP\UniBo\A1-RNN-SequenceLabeling


dependency_treebank.zip: 0.00B [00:00, ?B/s]

dependency_treebank.zip: 459kB [00:00, 1.49MB/s]                            

Download complete!
Extracting dataset... (it may take a while...)
Extracting zipfile
Extraction completed!


### Encode the corpus into a pandas.DataFrame object.

In [21]:
def count_elements_in_folder(directory_path: Path):
    # Get a list of elements in the directory
    elements = list(directory_path.iterdir())

    # Count the number of elements
    num_elements = len(elements)

    return num_elements

# Example usage:
folder_datasetdirectory_path = Path("datasets/dependency_treebank")
num_elements = count_elements_in_folder(folder_datasetdirectory_path)

print("Number of Elements:", num_elements)

Number of Elements: 199


In [25]:
def get_names_of_files_in_folder(directory_path: Path, num_elements: int = 10):
    # Get a list of the first 'num_elements' elements in the directory
    elements = sorted(directory_path.iterdir())[:num_elements]

    # Extract names from the elements
    names = [element.name for element in elements]

    return names

file_names = get_names_of_files_in_folder(folder_datasetdirectory_path, num_elements)

print(file_names)


['wsj_0001.dp', 'wsj_0002.dp', 'wsj_0003.dp', 'wsj_0004.dp', 'wsj_0005.dp', 'wsj_0006.dp', 'wsj_0007.dp', 'wsj_0008.dp', 'wsj_0009.dp', 'wsj_0010.dp', 'wsj_0011.dp', 'wsj_0012.dp', 'wsj_0013.dp', 'wsj_0014.dp', 'wsj_0015.dp', 'wsj_0016.dp', 'wsj_0017.dp', 'wsj_0018.dp', 'wsj_0019.dp', 'wsj_0020.dp', 'wsj_0021.dp', 'wsj_0022.dp', 'wsj_0023.dp', 'wsj_0024.dp', 'wsj_0025.dp', 'wsj_0026.dp', 'wsj_0027.dp', 'wsj_0028.dp', 'wsj_0029.dp', 'wsj_0030.dp', 'wsj_0031.dp', 'wsj_0032.dp', 'wsj_0033.dp', 'wsj_0034.dp', 'wsj_0035.dp', 'wsj_0036.dp', 'wsj_0037.dp', 'wsj_0038.dp', 'wsj_0039.dp', 'wsj_0040.dp', 'wsj_0041.dp', 'wsj_0042.dp', 'wsj_0043.dp', 'wsj_0044.dp', 'wsj_0045.dp', 'wsj_0046.dp', 'wsj_0047.dp', 'wsj_0048.dp', 'wsj_0049.dp', 'wsj_0050.dp', 'wsj_0051.dp', 'wsj_0052.dp', 'wsj_0053.dp', 'wsj_0054.dp', 'wsj_0055.dp', 'wsj_0056.dp', 'wsj_0057.dp', 'wsj_0058.dp', 'wsj_0059.dp', 'wsj_0060.dp', 'wsj_0061.dp', 'wsj_0062.dp', 'wsj_0063.dp', 'wsj_0064.dp', 'wsj_0065.dp', 'wsj_0066.dp', 'wsj_0067

In [41]:
dataframe_rows = []

for file_path in folder_datasetdirectory_path.glob('*.dp'):
    phrase = ""
    tokenized = []
    tagged = []
    #nouns = []
    #verbs = []
    #cardinal_digits = []
    with file_path.open(mode='r', encoding='utf-8') as text_file:
        for row in text_file:
            words = re.split(r'\s+', row)
            phrase += words[0] + " "
            tokenized.append(words[0])
            tagged.append((words[0],words[1]))
    
        dataframe_row = {
            "phrase": phrase,
            "tokenized": tokenized,
            "tagged": tagged
        }
        dataframe_rows.append(dataframe_row)

print(len(dataframe_rows))

199


In [ ]:
dataframe_name = ""

### Splits it in training, validation, and test sets.

The corpus contains 200 documents.

   * **Train**: Documents 1-100
   * **Validation**: Documents 101-150
   * **Test**: Documents 151-199

# T2 - Text encoding

To train a neural POS tagger, you first need to encode text into numerical format.

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

# T3 - Model definition

You are now tasked to define your neural POS tagger.

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

* **Do not mix Model 1 and Model 2**. Each model has its own instructions.

**Note**: if a document contains many tokens, you are **free** to split them into chunks or sentences to define your mini-batches.

# [Task 4 - 1.0 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using macro F1-score, compute over **all** tokens.
* **Concatenate** all tokens in a data split to compute the F1-score. (**Hint**: accumulate FP, TP, FN, TN iteratively) 
* **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

**Note**: What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe **must** be added to the vocabulary.
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **special token** (e.g., [UNK]) and a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)

### More about OOV

For a given token:

* **If in train set**: add to vocabulary and assign an embedding (use GloVe if token in GloVe, custom embedding otherwise).
* **If in val/test set**: assign special token if not in vocabulary and assign custom embedding. 

Your vocabulary **should**:

* Contain all tokens in train set; or
* Union of tokens in train set and in GloVe $\rightarrow$ we make use of existing knowledge!


### Token to embedding mapping

You can follow two approaches for encoding tokens in your POS tagger.

### Work directly with embeddings

- Compute the embedding of each input token
- Feed the mini-batches of shape (batch_size, # tokens, embedding_dim) to your model

### Work with Embedding layer

- Encode input tokens to token ids
- Define a Embedding layer as the first layer of your model
- Compute the embedding matrix of all known tokens (i.e., tokens in your vocabulary)
- Initialize the Embedding layer with the computed embedding matrix
- You are **free** to set the Embedding layer trainable or not

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim=vocab_size,
                                      output_dim=embedding_dimension,
                                      weights=[embedding_matrix],
                                      mask_zero=True,                   # automatically masks padding tokens
                                      name='encoder_embedding')

### Padding

Pay attention to padding tokens!

Your model **should not** be penalized on those tokens.

#### How to?

There are two main ways.

However, their implementation depends on the neural library you are using.

- Embedding layer
- Custom loss to compute average cross-entropy on non-padding tokens only

**Note**: This is a **recommendation**, but we **do not penalize** for missing workarounds.

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline, Model 1, and Model 2.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

# [Task 6 - 1.0 points] Error Analysis

You are tasked to evaluate your best performing model.

### Instructions

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible) 
* Comment the about errors and propose possible solutions on how to address them.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Execution Order

You are **free** to address tasks in any order (if multiple orderings are available).

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Robust Evaluation

Each model is trained with at least 3 random seeds.

Task 4 requires you to compute the average performance over the 3 seeds and its corresponding standard deviation.

### Model Selection for Analysis

To carry out the error analysis you are **free** to either

* Pick examples or perform comparisons with an individual seed run model (e.g., Baseline seed 1337)
* Perform ensembling via, for instance, majority voting to obtain a single model.

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Punctuation

**Do not** remove punctuation from documents since it may be helpful to the model.

You should **ignore** it during metrics computation.

If you are curious, you can run additional experiments to verify the impact of removing punctuation.